## WUMPUS WORLD AI SOLVER 

In [4]:
## Importing Library Pygame
%pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pygame
import random
import time
import math
from enum import Enum, auto
from collections import deque
import heapq

# Initialize pygame
pygame.init()

# Constants
GRID_SIZE = 4
CELL_SIZE = 100
WIDTH = GRID_SIZE * CELL_SIZE + 400  # Extra space for info panel
HEIGHT = GRID_SIZE * CELL_SIZE
FPS = 30

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
BROWN = (165, 42, 42)
DARK_GRAY = (50, 50, 50)
LIGHT_BLUE = (173, 216, 230)

# Direction enum
class Direction(Enum):
    UP = auto()
    RIGHT = auto()
    DOWN = auto()
    LEFT = auto()

# Create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Wumpus World")
clock = pygame.time.Clock()

# Load images
gold_img = pygame.Surface((CELL_SIZE-20, CELL_SIZE-20), pygame.SRCALPHA)
pygame.draw.circle(gold_img, YELLOW, (CELL_SIZE//2-10, CELL_SIZE//2-10), CELL_SIZE//3)

wumpus_img = pygame.Surface((CELL_SIZE-20, CELL_SIZE-20), pygame.SRCALPHA)
pygame.draw.polygon(wumpus_img, RED, [(CELL_SIZE//2-10, 10), (CELL_SIZE-30, CELL_SIZE-30), (10, CELL_SIZE-30)])

pit_img = pygame.Surface((CELL_SIZE-20, CELL_SIZE-20), pygame.SRCALPHA)
pygame.draw.circle(pit_img, BLACK, (CELL_SIZE//2-10, CELL_SIZE//2-10), CELL_SIZE//3)

agent_img = pygame.Surface((CELL_SIZE-40, CELL_SIZE-40), pygame.SRCALPHA)
pygame.draw.circle(agent_img, BLUE, (CELL_SIZE//2-20, CELL_SIZE//2-20), CELL_SIZE//4)

# Fonts
font = pygame.font.SysFont(None, 24)

class Cell:
    def __init__(self):
        self.has_wumpus = False
        self.has_pit = False
        self.has_gold = False
        self.has_breeze = False
        self.has_stench = False
        self.visited = False
        
    def is_dangerous(self):
        return self.has_wumpus or self.has_pit

class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.grid = [[Cell() for _ in range(size)] for _ in range(size)]
        self.agent_x, self.agent_y = 0, 3  # (1,1) in map coordinates, [3][0] in array
        self.agent_direction = Direction.RIGHT
        self.game_over = False
        self.victory = False
        self.score = 0
        self.moves = 0
        self.messages = []
        self.agent_has_gold = False  # New flag to track if agent has picked up gold
        
        # Initialize the world with the fixed map
        self.initialize_fixed_map()
    
    def initialize_fixed_map(self):
        # Clear existing grid
        self.grid = [[Cell() for _ in range(self.size)] for _ in range(self.size)]
        
        # Convert from the 1-indexed map coordinates to 0-indexed array coordinates
        # In the map: (x,y) where (1,1) is bottom-left and (4,4) is top-right
        # In our code: [y][x] where [0][0] is top-left and [3][3] is bottom-right
        
        # Place Wumpus at (1,3) in map coordinates -> [1][0] in array
        self.grid[1][0].has_wumpus = True
        
        # Place pits according to the map:
        # - (3,1) in map -> [3][2] in array
        # - (3,3) in map -> [1][2] in array
        # - (4,4) in map -> [0][3] in array
        self.grid[3][2].has_pit = True  # (3,1)
        self.grid[1][2].has_pit = True  # (3,3)
        self.grid[0][3].has_pit = True  # (4,4)
        
        # Place gold at (2,3) in map coordinates -> [1][1] in array
        self.grid[1][1].has_gold = True
        
        # Calculate breezes and stenches
        for y in range(self.size):
            for x in range(self.size):
                # If there's a pit, add breeze to adjacent cells
                if self.grid[y][x].has_pit:
                    for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.size and 0 <= ny < self.size:
                            self.grid[ny][nx].has_breeze = True
                
                # If there's a wumpus, add stench to adjacent cells
                if self.grid[y][x].has_wumpus:
                    for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.size and 0 <= ny < self.size:
                            self.grid[ny][nx].has_stench = True
        
        # Mark the starting position (1,1) in map -> [3][0] in array
        self.grid[3][0].visited = True
        
    def get_percepts(self, x, y):
        percepts = {
            "breeze": self.grid[y][x].has_breeze,
            "stench": self.grid[y][x].has_stench,
            "glitter": self.grid[y][x].has_gold,
            "bump": False,  # Will be set true if agent tries to move outside the grid
            "scream": False  # Will be set true if wumpus is killed
        }
        return percepts
    
    def move_agent(self, direction):
        if self.game_over:
            return False
        
        self.moves += 1
        self.agent_direction = direction
        
        # Calculate new position
        new_x, new_y = self.agent_x, self.agent_y
        if direction == Direction.UP:
            new_y -= 1
        elif direction == Direction.RIGHT:
            new_x += 1
        elif direction == Direction.DOWN:
            new_y += 1
        elif direction == Direction.LEFT:
            new_x -= 1
        
        # Check if new position is valid
        if new_x < 0 or new_x >= self.size or new_y < 0 or new_y >= self.size:
            self.add_message("Bump! Hit wall.")
            return False
        
        # Move agent
        self.agent_x, self.agent_y = new_x, new_y
        self.grid[new_y][new_x].visited = True
        
        # Check for game over conditions
        if self.grid[new_y][new_x].has_wumpus:
            self.game_over = True
            self.add_message("GAME OVER! Eaten by Wumpus!")
            self.score -= 1000
            return True
        
        if self.grid[new_y][new_x].has_pit:
            self.game_over = True
            self.add_message("GAME OVER! Fell into a pit!")
            self.score -= 1000
            return True
        
        # If agent has found gold and is back at start position, victory!
        if self.agent_has_gold and new_x == 0 and new_y == 3:  # Start position (1,1) in map
            self.victory = True
            self.game_over = True
            self.add_message("VICTORY! Returned to start with gold!")
            self.score += 1000
            return True
        
        # Pick up gold if present (without ending the game)
        if self.grid[new_y][new_x].has_gold:
            self.agent_has_gold = True
            self.grid[new_y][new_x].has_gold = False  # Remove gold from the cell
            self.add_message("Found gold! Now return to start.")
            self.score += 500  # Partial reward for finding gold
        
        # Cost of movement
        self.score -= 1
        return True
    
    def add_message(self, msg):
        self.messages.append(msg)
        if len(self.messages) > 5:
            self.messages.pop(0)

class AgentKnowledge:
    def __init__(self, world_size):
        self.size = world_size
        # Knowledge base: -1 = unknown, 0 = safe, 1 = danger
        self.kb = [[-1 for _ in range(world_size)] for _ in range(world_size)]
        self.visited = [[False for _ in range(world_size)] for _ in range(world_size)]
        self.wumpus_probability = [[0.0 for _ in range(world_size)] for _ in range(world_size)]
        self.pit_probability = [[0.0 for _ in range(world_size)] for _ in range(world_size)]
        self.gold_found = False
        
        # Mark starting position as safe (1,1) in map coordinates -> [3][0] in array
        self.kb[3][0] = 0
        self.visited[3][0] = True
    
    def update_knowledge(self, x, y, percepts):
        self.visited[y][x] = True
        self.kb[y][x] = 0  # Mark current cell as safe
        
        # If there's gold, mark it
        if percepts["glitter"]:
            self.gold_found = True
        
        # Update adjacent cells based on percepts
        if not percepts["breeze"] and not percepts["stench"]:
            # Mark adjacent cells as safe
            for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.size and 0 <= ny < self.size:
                    self.kb[ny][nx] = 0
        
        if percepts["breeze"]:
            # Increase pit probability in adjacent cells
            for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.size and 0 <= ny < self.size and not self.visited[ny][nx]:
                    self.pit_probability[ny][nx] += 0.2
                    if self.pit_probability[ny][nx] > 0.5:
                        self.kb[ny][nx] = 1  # Mark as dangerous
        
        if percepts["stench"]:
            # Increase wumpus probability in adjacent cells
            for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.size and 0 <= ny < self.size and not self.visited[ny][nx]:
                    self.wumpus_probability[ny][nx] += 0.2
                    if self.wumpus_probability[ny][nx] > 0.5:
                        self.kb[ny][nx] = 1  # Mark as dangerous

class WumpusAgent:
    def __init__(self, world_size):
        self.world_size = world_size
        self.knowledge = AgentKnowledge(world_size)
        self.current_x = 0
        self.current_y = world_size - 1  # (1,1) in map coordinates, [3][0] in array coordinates
        self.path = []
        self.has_gold = False
        self.returning_home = False
    
    def decide_move(self, percepts):
        # Update knowledge with new percepts
        self.knowledge.update_knowledge(self.current_x, self.current_y, percepts)
        
        # Look for gold
        if percepts["glitter"]:
            self.has_gold = True
            self.returning_home = True
            # Return to start if we have the gold
            self.path = self.find_path_to((0, 3))  # Path to (1,1) in map coordinates
            if self.path:
                self.path.pop(0)  # Remove current position from path
        
        # If we're already returning home, prioritize that path
        if self.returning_home and not self.path:
            self.path = self.find_path_to((0, 3))
            if self.path:
                self.path.pop(0)  # Remove current position from path
                
        # Plan path if we don't have one
        if not self.path and not self.returning_home:
            self.plan_path()
        
        # If still no path, try risky moves
        if not self.path and not self.returning_home:
            self.plan_risky_path()
        
        # Execute move if we have a path
        if self.path:
            next_x, next_y = self.path.pop(0)
            
            # Determine the direction to move
            if next_x > self.current_x:
                move = Direction.RIGHT
            elif next_x < self.current_x:
                move = Direction.LEFT
            elif next_y > self.current_y:
                move = Direction.DOWN
            else:
                move = Direction.UP
            
            # Update agent's position
            self.current_x, self.current_y = next_x, next_y
            return move
        
        # If no path is found, move randomly (should rarely happen)
        return random.choice([Direction.UP, Direction.RIGHT, Direction.DOWN, Direction.LEFT])
    
    def find_path_to(self, target):
        # Use A* to find path to target
        start = (self.current_x, self.current_y)
        frontier = [(0, start, [])]  # (priority, position, path)
        visited = set([start])
        
        while frontier:
            _, (x, y), path = heapq.heappop(frontier)
            
            if (x, y) == target:
                return path + [(x, y)] if (x, y) != start else path
            
            # Expand to neighbors
            for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if (0 <= nx < self.world_size and 
                    0 <= ny < self.world_size and 
                    (nx, ny) not in visited and 
                    self.knowledge.kb[ny][nx] != 1):  # Not known to be dangerous
                    
                    # Manhattan distance heuristic
                    h = abs(nx - target[0]) + abs(ny - target[1])
                    new_path = path + [(nx, ny)]
                    priority = len(new_path) + h
                    
                    heapq.heappush(frontier, (priority, (nx, ny), new_path))
                    visited.add((nx, ny))
        
        return []  # No path found
    
    def plan_path(self):
        # Look for unvisited safe cells
        targets = []
        for y in range(self.world_size):
            for x in range(self.world_size):
                if not self.knowledge.visited[y][x] and self.knowledge.kb[y][x] == 0:
                    targets.append((x, y))
        
        # Find the closest one
        if targets:
            best_path = None
            for target in targets:
                path = self.find_path_to(target)
                if path and (best_path is None or len(path) < len(best_path)):
                    best_path = path
            
            if best_path:
                self.path = best_path
                return
    
    def plan_risky_path(self):
        # Find least risky unvisited cell
        min_risk = float('inf')
        target = None
        
        for y in range(self.world_size):
            for x in range(self.world_size):
                if not self.knowledge.visited[y][x]:
                    risk = self.knowledge.pit_probability[y][x] + self.knowledge.wumpus_probability[y][x]
                    if risk < min_risk:
                        min_risk = risk
                        target = (x, y)
        
        if target:
            # Find path to target using A*
            self.path = self.find_path_to(target)

def draw_grid(world):
    # Draw grid
    for y in range(world.size):
        for x in range(world.size):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            
            # Draw cell background
            if world.grid[y][x].visited:
                pygame.draw.rect(screen, LIGHT_BLUE, rect)
            else:
                pygame.draw.rect(screen, WHITE, rect)
            pygame.draw.rect(screen, BLACK, rect, 1)
            
            # Draw grid coordinates - adjusted to match map coordinates (1,1) at bottom-left
            coord_text = font.render(f"({x+1},{4-y})", True, BLACK)
            screen.blit(coord_text, (x * CELL_SIZE + 5, y * CELL_SIZE + 5))
            
            # Draw cell contents if visited or game is over
            if world.grid[y][x].visited or world.game_over:
                # Draw pit
                if world.grid[y][x].has_pit:
                    screen.blit(pit_img, (x * CELL_SIZE + 10, y * CELL_SIZE + 10))
                    pit_text = font.render("PIT", True, WHITE)
                    screen.blit(pit_text, (x * CELL_SIZE + 40, y * CELL_SIZE + 40))
                
                # Draw wumpus
                if world.grid[y][x].has_wumpus:
                    screen.blit(wumpus_img, (x * CELL_SIZE + 10, y * CELL_SIZE + 10))
                    wumpus_text = font.render("WUMPUS", True, RED)
                    screen.blit(wumpus_text, (x * CELL_SIZE + 25, y * CELL_SIZE + 40))
                
                # Draw gold
                if world.grid[y][x].has_gold:
                    screen.blit(gold_img, (x * CELL_SIZE + 10, y * CELL_SIZE + 10))
                    gold_text = font.render("GOLD", True, BLACK)
                    screen.blit(gold_text, (x * CELL_SIZE + 35, y * CELL_SIZE + 40))
                
                # Draw percepts
                if world.grid[y][x].has_breeze:
                    breeze_text = font.render("Breeze", True, BLUE)
                    screen.blit(breeze_text, (x * CELL_SIZE + 5, y * CELL_SIZE + 60))
                
                if world.grid[y][x].has_stench:
                    stench_text = font.render("Stench", True, BROWN)
                    screen.blit(stench_text, (x * CELL_SIZE + 5, y * CELL_SIZE + 80))
    
    # Mark start position
    start_text = font.render("START", True, GREEN)
    screen.blit(start_text, (0 * CELL_SIZE + 30, 3 * CELL_SIZE + 40))
    
    # Draw agent
    agent_rect = agent_img.get_rect(center=((world.agent_x + 0.5) * CELL_SIZE, (world.agent_y + 0.5) * CELL_SIZE))
    screen.blit(agent_img, agent_rect)
    
    # Indicate if agent has gold
    if world.agent_has_gold:
        gold_indicator = font.render("HAS GOLD", True, YELLOW)
        screen.blit(gold_indicator, (world.agent_x * CELL_SIZE + 10, world.agent_y * CELL_SIZE + 10))

def draw_info_panel(world, agent):
    # Draw info panel background
    panel_rect = pygame.Rect(world.size * CELL_SIZE, 0, 400, HEIGHT)
    pygame.draw.rect(screen, GRAY, panel_rect)
    
    # Draw score and moves
    score_text = font.render(f"Score: {world.score}", True, BLACK)
    screen.blit(score_text, (world.size * CELL_SIZE + 20, 20))
    
    moves_text = font.render(f"Moves: {world.moves}", True, BLACK)
    screen.blit(moves_text, (world.size * CELL_SIZE + 20, 50))
    
    # Draw game status
    if world.game_over:
        if world.victory:
            status_text = font.render("STATUS: VICTORY!", True, GREEN)
        else:
            status_text = font.render("STATUS: GAME OVER", True, RED)
    else:
        if world.agent_has_gold:
            status_text = font.render("STATUS: RETURNING HOME WITH GOLD", True, YELLOW)
        else:
            status_text = font.render("STATUS: EXPLORING", True, BLUE)
    screen.blit(status_text, (world.size * CELL_SIZE + 20, 80))
    
    # Draw agent knowledge
    kb_text = font.render("Agent Knowledge:", True, BLACK)
    screen.blit(kb_text, (world.size * CELL_SIZE + 20, 120))
    
    # Draw minimap of agent's knowledge
    minimap_size = 20
    for y in range(agent.world_size):
        for x in range(agent.world_size):
            mini_rect = pygame.Rect(
                world.size * CELL_SIZE + 20 + x * minimap_size, 
                150 + y * minimap_size, 
                minimap_size, minimap_size
            )
            
            # Color based on knowledge
            if agent.knowledge.visited[y][x]:
                color = GREEN
            elif agent.knowledge.kb[y][x] == 0:
                color = LIGHT_BLUE  # Safe but not visited
            elif agent.knowledge.kb[y][x] == 1:
                color = RED  # Dangerous
            else:
                color = DARK_GRAY  # Unknown
            
            pygame.draw.rect(screen, color, mini_rect)
            pygame.draw.rect(screen, BLACK, mini_rect, 1)
    
    # Draw minimap coordinates
    mini_coord_text = font.render("Agent Knowledge Map:", True, BLACK)
    screen.blit(mini_coord_text, (world.size * CELL_SIZE + 20, 120))
    
    # Draw legend
    legend_y = 240
    legend_titles = [
        ("Visited", GREEN),
        ("Known Safe", LIGHT_BLUE),
        ("Known Dangerous", RED),
        ("Unknown", DARK_GRAY)
    ]
    
    for title, color in legend_titles:
        pygame.draw.rect(screen, color, (world.size * CELL_SIZE + 20, legend_y, 20, 20))
        pygame.draw.rect(screen, BLACK, (world.size * CELL_SIZE + 20, legend_y, 20, 20), 1)
        legend_text = font.render(title, True, BLACK)
        screen.blit(legend_text, (world.size * CELL_SIZE + 50, legend_y))
        legend_y += 30
    
    # Draw current percepts
    percepts = world.get_percepts(world.agent_x, world.agent_y)
    percepts_y = 350
    percepts_text = font.render("Current Percepts:", True, BLACK)
    screen.blit(percepts_text, (world.size * CELL_SIZE + 20, percepts_y))
    percepts_y += 30
    
    for percept, value in percepts.items():
        if value:
            p_text = font.render(f"- {percept.capitalize()}", True, BLUE)
            screen.blit(p_text, (world.size * CELL_SIZE + 20, percepts_y))
            percepts_y += 20
    
    # Draw messages
    msg_y = max(percepts_y + 20, 450)
    msg_text = font.render("Messages:", True, BLACK)
    screen.blit(msg_text, (world.size * CELL_SIZE + 20, msg_y))
    msg_y += 30
    
    for msg in world.messages:
        msg_text = font.render(msg, True, BLACK)
        screen.blit(msg_text, (world.size * CELL_SIZE + 20, msg_y))
        msg_y += 25

def main():
    # Create world and agent
    world = WumpusWorld(GRID_SIZE)
    agent = WumpusAgent(GRID_SIZE)
    
    # Main game loop
    running = True
    auto_play = False
    move_delay = 0.5  # seconds between automatic moves
    last_move_time = 0
    
    while running:
        current_time = time.time()
        
        # Process input
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    # Reset game
                    world = WumpusWorld(GRID_SIZE)
                    agent = WumpusAgent(GRID_SIZE)
                    auto_play = False
                elif event.key == pygame.K_SPACE:
                    # Toggle auto-play
                    auto_play = not auto_play
                    last_move_time = current_time
                    if auto_play:
                        world.add_message("Auto-solver activated")
                    else:
                        world.add_message("Auto-solver paused")
                elif event.key == pygame.K_UP:
                    world.move_agent(Direction.UP)
                elif event.key == pygame.K_RIGHT:
                    world.move_agent(Direction.RIGHT)
                elif event.key == pygame.K_DOWN:
                    world.move_agent(Direction.DOWN)
                elif event.key == pygame.K_LEFT:
                    world.move_agent(Direction.LEFT)
                elif event.key == pygame.K_s:
                    # Make one step in auto-play
                    if not world.game_over:
                        percepts = world.get_percepts(world.agent_x, world.agent_y)
                        move = agent.decide_move(percepts)
                        world.move_agent(move)
                        world.add_message(f"Agent moved: {move.name}")
        
        # Auto-play logic
        if auto_play and not world.game_over and current_time - last_move_time >= move_delay:
            percepts = world.get_percepts(world.agent_x, world.agent_y)
            move = agent.decide_move(percepts)
            world.move_agent(move)
            world.add_message(f"Agent moved: {move.name}")
            last_move_time = current_time
        
        # Draw everything
        screen.fill(WHITE)
        draw_grid(world)
        draw_info_panel(world, agent)
        
        # Update display
        pygame.display.flip()
        clock.tick(FPS)
    
    pygame.quit()

if __name__ == "__main__":
    main()